In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('311_Customer_Service_Requests.csv')

D:\ProgramFiles\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,SRRecordID,ServiceRequesNum,SRType,Agency,Neighborhood,StreetAddress,ZipCode,MethodReceived,SRStatus,CreatedDate,StatusDate,DueDate,LastActivity,Outcome,LastActivityDate,GeoLocation
0,1094344940,15-00072082,WW Water Leak (Exterior),Bureau of Water and Waste Water,CANTON,1013 S CLINTON ST,21224,Interface,CLOSED,02/01/2015 08:12:00 AM +0000,02/01/2015 06:23:00 PM +0000,02/03/2015 08:12:00 AM +0000,Cityworks SR Comments,UPDATED,NaN,NaN
1,1094344982,15-00072083,BCLB-Liquor License Complaint,Liquor License Board,GREEKTOWN,500 S LEHIGH ST,21224,Interface,CLOSED,02/01/2015 08:48:00 AM +0000,02/18/2015 09:12:00 PM +0000,03/21/2015 07:48:00 AM +0000,Dispatch Investigator,Closed,NaN,NaN
2,1094345013,15-00072084,WW Hydrant Open,Bureau of Water and Waste Water,MILLHILL,2600 WILKENS AVE,21223,Phone,HOLDLOCK,02/01/2015 09:13:00 AM +0000,02/01/2015 11:07:00 AM +0000,02/02/2015 09:13:00 AM +0000,Cityworks SR Comments,UPDATED,NaN,NaN
3,1094345094,15-00072085,TRM-Snow/Icy Conditions,Department of Transportation,MEDFIELD,4496 83 N,21209,Phone,CLOSED,02/01/2015 09:24:00 AM +0000,02/04/2015 10:41:00 PM +0000,02/02/2015 09:24:00 AM +0000,Cityworks SR Comments,UPDATED,NaN,NaN
4,1094345124,15-00072086,TRM-Snow/Icy Conditions,Department of Transportation,EAST BALTIMORE MIDWAY,732 BARTLETT AVE,21218,Phone,CLOSED,02/01/2015 09:29:00 AM +0000,02/03/2015 09:09:00 PM +0000,02/02/2015 09:29:00 AM +0000,Cityworks SR Comments,UPDATED,NaN,NaN


In [ ]:
df.SRType.fillna('NaN', inplace=True)

In [5]:
srtype = df.SRType.value_counts().to_frame()
srtype.columns = ['count']
srtype['percentage'] = srtype['count']/srtype['count'].sum()
# srtype

In [6]:
df['TypePrefixLen'] = df.SRType.apply(lambda x: x.split('-')[0].split(' ').__len__())

df['TypePrefix'] = df.SRType.apply(lambda x: x.split('-')[0].split(' ')[0])

df[df.TypePrefixLen!=1].TypePrefix.value_counts()

pa = df.groupby(['TypePrefix', 'Agency']).count()[['SRRecordID']]

pa.columns = ['count']

pa

count
TypePrefix  Agency                                           
AAA         Mayors Office of Information Technology       641
BCLB        Liquor License Board                         2645
BGE         Department of Transportation                47560
BPD         Baltimore Police Department                   310
CCTV        Mayors Office of Information Technology        15
DPW         Bureau of Solid Waste                         487
            Department of General Services                663
            Department of Public Works                     94
ECC         Mayors Office of Information Technology     73002
FC          Finance-Revenue Collections                     2
FCCS        Finance-Revenue Collections                 19022
FCDA        Finance-Revenue Collections                  1036
FCPF        Finance-Revenue Collections                     5
FIN         Department of Finance                        1269
FINBAPS     Department of Finance                          48
FIR         Fire Department                             30389
Forestry    Department of Recreation and Parks          29530
GS          Department of General Services                275
            Department of Transportation                  450
GSI         Department of General Services                  6
HCD         Fire Department                                 5
            HCD-Department of Housing                  260631
HLTH        Health Department                           65494
MOHS        Mayors Office                                 352
MOIT        Mayors Office of Information Technology        94
MONCS       Mayors Office                                 200
Mayor's     Mayors Office                                 978
NaN         Bureau of Solid Waste                        4101
            Bureau of Water and Waste Water               430
            Department of Transportation                 5892
            Finance-Revenue Collections                     1
            Health Department                              90
PABC        Parking Authority of Baltimore City Inc.     4290
RP          Department of Recreation and Parks           7713
SW          Bureau of Solid Waste                     1010109
TEC         Department of Transportation                19325
TR          Department of Transportation                 5416
TRA         Department of Transportation                  166
TRC         Department of Transportation                 1594
TRD         Department of Transportation                   19
TRM         Department of Transportation               132198
TRS         Department of Transportation               159635
TRT         Department of Transportation                27726
TTR         Department of Transportation                 6301
WW          Bureau of Water and Waste Water            161454
            Department of Public Works                    235
            Finance-Revenue Collections                     1
zzECC       Mayors Office of Information Technology         1
zzTRM       Department of Transportation                   12
zzz         Department of Recreation and Parks             20
zzzForestry Mayors Office of Information Technology         6
zzzTRT      Department of Transportation                    1
zzzWW       Bureau of Water and Waste Water                 7